In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('hotData.csv')


In [3]:
y = df['results']
X = df.drop(columns=['results'])

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.10, random_state=2)

In [4]:
def cross_val(model):
    pred = cross_val_score(model, X_train, y_train, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [13]:
model = ElasticNet(alpha=0, tol=10)

In [14]:
model.fit(X_train, y_train)

/var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/ipykernel_77939/180087699.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train, y_train)
/Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0, tol=10)

In [15]:
test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

In [16]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

In [17]:
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))
# print('MAPE: '+str(mean_absolute_percentage_error(y_train, train_pred)))
# alpha=0, tol=10 MAPE: 0.4381142743608524


MAPE: 0.6790582811959258


In [10]:
# mape: 1.4778
# alpha=0 MAPE: 0.5071599152907355


In [11]:
# MAPE: 1.4778202775570333
# alpha=0 MAPE: 0.5071599152907355
#  alpha = 0 is equivalent to an ordinary least square, 
#     solved by the LinearRegression object.

In [12]:
print('ELASTIC NET')
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')

results_df = pd.DataFrame(data=[["Elastic Net Regression", *evaluate(y_test, test_pred) , cross_val(ElasticNet())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df, ignore_index=True)

ELASTIC NET
Test set evaluation:
_____________________________________
MAE: 5.565747808581173
MSE: 45.37736708048586
RMSE: 6.73627249155539
R2 Square 0.21068832554355443
__________________________________


In [18]:
data = {'y_test': y_test, 'y_pred': test_pred}
new_df = pd.DataFrame(data)
diff = (new_df['y_test'] - new_df['y_pred']).abs()
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)


In [19]:
new_df

,y_test,y_pred,diff
227,9,9.578111,0.578111
176,13,13.651812,0.651812
13,20,20.753064,0.753064
99,21,22.344676,1.344676
127,18,16.540084,1.459916
66,22,23.827563,1.827563
249,9,10.955498,1.955498
77,8,10.679230,2.679230
89,18,20.744533,2.744533
93,29,25.975019,3.024981


In [115]:
new_df['diff'].mean()

4.976872818396645

In [116]:
new_df['diff'].median()

3.595215189519795